In [1]:
from keras.layers import Flatten
from keras.layers import ConvLSTM2D
from keras.layers import concatenate

In [5]:
from numpy import array
from numpy import hstack 

In [6]:
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense 
from keras.layers import Reshape

In [9]:
import numpy as np
import pandas as pd 
import re 
import matplotlib.pyplot as plt 
import os
import gc
import joblib

In [10]:
from sklearn.model_selection import RandomizedSearchCV 

In [14]:
from keras.utils import to_categorical
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from sklearn.preprocessing import StandardScaler
import time 
from sklearn.metrics import fbeta_score
from sklearn.metrics import r2_score 
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

In [16]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor

In [17]:
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.optimizers import Adam 
from keras.constraints import maxnorm 
from keras.layers import Dropout 

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

In [20]:
import pickle
import tensorflow as tf

In [24]:
a = np.arange(15)
a.reshape(3,5)

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

In [29]:
x = np.mean(a.reshape(3,5), axis=0).reshape((1,5))

In [34]:
y = np.random.randn(5).reshape((1,5))

In [35]:
np.append(x,y, axis=0)

array([[ 5.        ,  6.        ,  7.        ,  8.        ,  9.        ],
       [-0.23841298,  0.04732875, -0.48385626, -2.01536364,  0.39870941]])

In [28]:
np.mean(a.reshape(3,5), axis=0)

array([5., 6., 7., 8., 9.])

In [37]:
input_dim=5
n_steps_in=7
n_features=13
n_steps_out=4
neurons=1
learn_rate=0.01
dropout_rate=0.0
weight_constraint=0
activation='sigmoid'
totalsets = 2
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Reshape(target_shape=(
    n_steps_in*totalsets, n_features), input_shape=(n_steps_in*n_features*totalsets,)))
model.add(tf.keras.layers.LSTM(neurons, activation=activation, return_sequences=True,
                               kernel_constraint=tf.keras.constraints.MaxNorm(weight_constraint)))

model.add(tf.keras.layers.Dropout(dropout_rate))
# , return_sequences=True))
model.add(tf.keras.layers.LSTM(neurons, activation=activation))
# model.add(Dense(neurons, activation=activation))  # Adding new layer
model.add(tf.keras.layers.Dense(n_steps_out))
opt = tf.keras.optimizers.Adam(lr=learn_rate)
model.compile(loss='mse', optimizer=opt)
# model.save(save_weights_only=True, best_model_only=True)
print('model: ' + str(model))

C:\Users\FangG\AppData\Roaming\Python\Python38\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


model: <keras.engine.sequential.Sequential object at 0x0000024C3D3978E0>


In [39]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 14, 13)            0         
                                                                 
 lstm (LSTM)                 (None, 14, 1)             60        
                                                                 
 dropout (Dropout)           (None, 14, 1)             0         
                                                                 
 lstm_1 (LSTM)               (None, 1)                 12        
                                                                 
 dense (Dense)               (None, 4)                 8         
                                                                 
Total params: 80
Trainable params: 80
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.layers

In [43]:
from keras.optimizers import Adam

In [44]:
model = Sequential()
model.add(Reshape(target_shape=(
    n_steps_in*totalsets, n_features), input_shape=(n_steps_in*n_features*totalsets,)))
model.add(LSTM(neurons, activation=activation,
               input_shape=(n_steps_in*totalsets, n_features)))
model.add(RepeatVector(1))
model.add(LSTM(neurons, activation=activation, return_sequences=True))
model.add(TimeDistributed(Dense(n_steps_out)))
opt = Adam(lr=learn_rate)
model.compile(loss='mse', optimizer=opt)

C:\Users\FangG\AppData\Roaming\Python\Python38\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [55]:
for l0 in model.layers:
    print(l0.name , l0.input_shape, l0.output_shape)

reshape_2 (None, 182) (None, 14, 13)
lstm_4 (None, 14, 13) (None, 1)
repeat_vector_1 (None, 1) (None, 1, 1)
lstm_5 (None, 1, 1) (None, 1, 1)
time_distributed_1 (None, 1, 1) (None, 1, 4)


In [45]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 14, 13)            0         
                                                                 
 lstm_4 (LSTM)               (None, 1)                 60        
                                                                 
 repeat_vector_1 (RepeatVect  (None, 1, 1)             0         
 or)                                                             
                                                                 
 lstm_5 (LSTM)               (None, 1, 1)              12        
                                                                 
 time_distributed_1 (TimeDis  (None, 1, 4)             8         
 tributed)                                                       
                                                                 
Total params: 80
Trainable params: 80
Non-trainable pa

In [56]:
#def create_reg_ConvLSTM_model(input_dim, n_steps_in, n_features, n_steps_out, neurons=1, learn_rate=0.01, activation='sigmoid'):
# reshape from [samples, timesteps] into [samples, timesteps, rows, columns, features(channels)]
model = Sequential()
model.add(Reshape(target_shape=(
    n_steps_in, totalsets, n_features, 1), input_shape=(n_steps_in*n_features*totalsets,)))
model.add(ConvLSTM2D(64, (1, 3), activation=activation,
                     input_shape=(n_steps_in, totalsets, n_features, 1)))
model.add(Flatten())
model.add(RepeatVector(1))
model.add(LSTM(neurons, activation=activation, return_sequences=True))
model.add(TimeDistributed(Dense(100, activation=activation)))
model.add(TimeDistributed(Dense(n_steps_out)))
opt = Adam(lr=learn_rate)
model.compile(loss='mse', optimizer=opt)

C:\Users\FangG\AppData\Roaming\Python\Python38\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [60]:
for l0 in model.layers:
    print(l0.__repr__ , l0.input_shape, l0.output_shape)

<method-wrapper '__repr__' of Reshape object at 0x0000024C44B28070> (None, 182) (None, 7, 2, 13, 1)
<method-wrapper '__repr__' of ConvLSTM2D object at 0x0000024C44B724C0> (None, 7, 2, 13, 1) (None, 2, 11, 64)
<method-wrapper '__repr__' of Flatten object at 0x0000024C44B8CE80> (None, 2, 11, 64) (None, 1408)
<method-wrapper '__repr__' of RepeatVector object at 0x0000024C44B704C0> (None, 1408) (None, 1, 1408)
<method-wrapper '__repr__' of LSTM object at 0x0000024C44B6AF70> (None, 1, 1408) (None, 1, 1)
<method-wrapper '__repr__' of TimeDistributed object at 0x0000024C44B47130> (None, 1, 1) (None, 1, 100)
<method-wrapper '__repr__' of TimeDistributed object at 0x0000024C44AA6A60> (None, 1, 100) (None, 1, 4)


In [63]:
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Reshape
from keras.models import Sequential
from keras.layers import TimeDistributed

In [67]:
l = TimeDistributed(Convolution2D(16, (3, 3), padding='same'), input_shape=(None, 28, 28, 1))


In [70]:
#I have 950 training video samples and 50 testing video samples. Each video sample has 10 frames and each frame has a shape of (n_row=28, n_col=28, n_channels=1). My inputs (x) and outputs (y) have same shapes.
"""
x_train shape: (950, 10, 28, 28,1),
y_train shape: (950, 10, 28, 28,1),
I want to give input video samples (x) as input to my model to predict output video samples (y).
"""
model = Sequential()
model.add(TimeDistributed(Convolution2D(16, (3, 3), padding='same', activation='sigmoid'), input_shape=(None, 28, 28, 1))) 
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(Dropout(0.2))
model.add(TimeDistributed(Convolution2D(32, (3, 3), padding='same', activation='sigmoid'))) 
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(Dropout(0.2))
model.add(TimeDistributed(Convolution2D(64, (3, 3), padding='same', activation='sigmoid'))) 
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(64, return_sequences=True, stateful=False))
model.add(LSTM(64, return_sequences=True, stateful=False, activation='sigmoid'))
model.add(Dense(784, activation='sigmoid'))
model.add(Reshape((-1, 28,28,1)))
model.compile(loss='mean_squared_error', optimizer='rmsprop')
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_22 (TimeDi  (None, None, 28, 28, 16)  160      
 stributed)                                                      
                                                                 
 time_distributed_23 (TimeDi  (None, None, 14, 14, 16)  0        
 stributed)                                                      
                                                                 
 dropout_5 (Dropout)         (None, None, 14, 14, 16)  0         
                                                                 
 time_distributed_24 (TimeDi  (None, None, 14, 14, 32)  4640     
 stributed)                                                      
                                                                 
 time_distributed_25 (TimeDi  (None, None, 7, 7, 32)   0         
 stributed)                                           

In [ ]:
# from keras.layers import Dense, Dropout, Activation, LSTM 
from tensorflow.keras.layers import BatchNormalization
from keras.layers import Lambda, Convolution2D, MaxPooling2D, Flatten, Reshape, Conv2D
from keras.layers.convolutional import Conv3D
from keras.models import Sequential
from keras.layers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling1D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Model
import keras.backend as K
import pylab as plt

In [74]:
MaxPooling2D?

In [73]:
model = Sequential()
model.add(TimeDistributed(Convolution2D(16, (3, 3), activation='relu', 
            kernel_initializer='glorot_uniform', padding='same'), input_shape=(10, 28, 28, 1))) 
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Convolution2D(32, (3,3), activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))))
model.add(Dropout(0.3))

model.add(TimeDistributed(Convolution2D(32, (3,3), activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Convolution2D(32, (3,3), activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))))
model.add(Dropout(0.3))

model.add(TimeDistributed(Convolution2D(32, (3,3), activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Convolution2D(32, (3,3), activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(1, 1))))
model.add(Dropout(0.3))

# extract features and dropout 
model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.3))
model.add(Dense(784, activation='linear'))
model.add(TimeDistributed(BatchNormalization()))

# input to LSTM
model.add(LSTM(units=512, activation='tanh', recurrent_activation='hard_sigmoid', kernel_initializer='glorot_uniform', unit_forget_bias=True, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(LSTM(units=512, activation='tanh', recurrent_activation='hard_sigmoid', kernel_initializer='glorot_uniform', unit_forget_bias=True, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))

# classifier with sigmoid activation for multilabel
model.add(Dense(784, activation='linear'))
# model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Reshape((28,28,1))))
model.compile(loss='mae', optimizer='rmsprop')
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_29 (TimeDi  (None, 10, 28, 28, 16)   160       
 stributed)                                                      
                                                                 
 time_distributed_30 (TimeDi  (None, 10, 28, 28, 16)   64        
 stributed)                                                      
                                                                 
 time_distributed_31 (TimeDi  (None, 10, 26, 26, 32)   4640      
 stributed)                                                      
                                                                 
 time_distributed_32 (TimeDi  (None, 10, 26, 26, 32)   128       
 stributed)                                                      
                                                                 
 time_distributed_33 (TimeDi  (None, 10, 25, 25, 32)  

In [ ]:
model = Sequential()
model.add(Reshape(target_shape=(
    n_steps_in, totalsets, n_features, 1), input_shape=(n_steps_in*n_features*totalsets,)))
model.add(ConvLSTM2D(64, (1, 3), activation=activation,
                     input_shape=(n_steps_in, totalsets, n_features, 1)))
model.add(Flatten())
model.add(RepeatVector(1))
model.add(LSTM(neurons, activation=activation, return_sequences=True))
model.add(TimeDistributed(Dense(100, activation=activation)))
model.add(TimeDistributed(Dense(n_steps_out)))
opt = Adam(lr=learn_rate)
model.compile(loss='mse', optimizer=opt)
return model

In [82]:
model = Sequential()
model.add(Reshape(target_shape=(
    n_steps_in, totalsets, n_features, 1), input_shape=(n_steps_in*n_features*totalsets,)))
model.add(ConvLSTM2D(64, (1, 3), activation=activation,
                     input_shape=(n_steps_in, totalsets, n_features, 1)))
model.add(Flatten())
model.add(RepeatVector(1))
opt = Adam(lr=learn_rate)
model.add(LSTM(neurons, activation=activation, return_sequences=True))
model.compile(loss='mse', optimizer=opt)
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_14 (Reshape)        (None, 7, 2, 13, 1)       0         
                                                                 
 conv_lstm2d_7 (ConvLSTM2D)  (None, 2, 11, 64)         50176     
                                                                 
 flatten_11 (Flatten)        (None, 1408)              0         
                                                                 
 repeat_vector_4 (RepeatVect  (None, 1, 1408)          0         
 or)                                                             
                                                                 
 lstm_15 (LSTM)              (None, 1, 1)              5640      
                                                                 
Total params: 55,816
Trainable params: 55,816
Non-trainable params: 0
_________________________________________________

In [ ]:
model = KerasRegressor(build_fn=create_reg_endecodeLSTM_model, input_dim=input_dim, activation=activation,
                                   epochs=epochs, batch_size=batch_size,  neurons=neurons, 
                       n_steps_in=int(n_steps_in), learn_rate=learn_rate, 
                       n_features=int(n_features), n_steps_out=int(n_steps_out), 
                       verbose=0)